# UAP Explorer - Spatiotemporal Baseline & Anomaly Detection

This notebook creates a baseline model for expected sighting counts per grid cell and time period,
then identifies spatiotemporal anomalies.

## Objectives
1. Aggregate sightings by spatial grid and time windows
2. Engineer features for baseline prediction
3. Train a model to predict expected sighting counts
4. Compute anomaly scores (z-scores/residuals)
5. Export grid-level anomaly scores for frontend

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load Cleaned Data

In [ ]:
# Load cleaned data
data_path = Path('../data/processed/cleaned_sightings.parquet')

if not data_path.exists():
    raise FileNotFoundError(
        f"Cleaned data not found at {data_path}.\n"
        "Please run scripts/clean_data.py first."
    )

df = pd.read_parquet(data_path)

print(f"✓ Loaded {len(df):,} cleaned sightings")
print(f"✓ Date range: {df['year'].min():.0f} - {df['year'].max():.0f}")
print(f"✓ Grid cells: {df['grid_id'].nunique():,}")
print(f"\nDataset shape: {df.shape}")
df.head()

## 2. Spatiotemporal Aggregation

Aggregate sightings by:
- **Spatial**: 1° x 1° grid cells (already computed as grid_id)
- **Temporal**: Monthly windows (year_month)

In [ ]:
# Convert year_month from Period to string for aggregation
df['year_month_str'] = df['year_month'].astype(str)

# Aggregate by grid cell and month
aggregated = df.groupby(['grid_id', 'grid_lat', 'grid_lon', 'year', 'month', 'year_month_str']).agg({
    'id': 'count',  # Count of sightings
}).reset_index()

aggregated.columns = ['grid_id', 'grid_lat', 'grid_lon', 'year', 'month', 'year_month', 'count']

print(f"✓ Aggregated to {len(aggregated):,} grid-time cells")
print(f"✓ Unique grid cells: {aggregated['grid_id'].nunique():,}")
print(f"✓ Time range: {aggregated['year'].min():.0f}-{aggregated['year'].max():.0f}")
print(f"\nSample aggregated data:")
aggregated.head(10)

In [ ]:
# Visualize distribution of counts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of counts
axes[0].hist(aggregated['count'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Sightings per Grid-Month')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Sighting Counts')
axes[0].axvline(aggregated['count'].median(), color='red', linestyle='--', label=f'Median: {aggregated["count"].median():.0f}')
axes[0].legend()

# Log-scale histogram
axes[1].hist(aggregated['count'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Sightings per Grid-Month')
axes[1].set_ylabel('Frequency (log scale)')
axes[1].set_title('Distribution of Sighting Counts (Log Scale)')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

print(f"\nCount statistics:")
print(aggregated['count'].describe())

## 3. Feature Engineering for Baseline Model

Create features to predict expected sighting counts:
- Temporal: year, month, seasonality
- Spatial: grid location
- Historical: lagged counts (autoregressive)

In [ ]:
# Create a complete time series for each grid cell
# This ensures we have entries for all months, even if count is 0

# Get all unique combinations of grid cells and months
all_grids = aggregated['grid_id'].unique()
all_months = pd.date_range(
    start=f"{aggregated['year'].min()}-{aggregated['month'].min():02d}-01",
    end=f"{aggregated['year'].max()}-{aggregated['month'].max():02d}-01",
    freq='MS'
)

# For simplicity, we'll work with the existing aggregated data
# In production, you might want to fill missing months with 0 counts

# Add seasonality features
aggregated['month_sin'] = np.sin(2 * np.pi * aggregated['month'] / 12)
aggregated['month_cos'] = np.cos(2 * np.pi * aggregated['month'] / 12)

# Add year trend (normalized)
year_min = aggregated['year'].min()
year_max = aggregated['year'].max()
aggregated['year_normalized'] = (aggregated['year'] - year_min) / (year_max - year_min)

# Add historical count features (lagged)
# Sort by grid and time
aggregated = aggregated.sort_values(['grid_id', 'year', 'month'])

# Create lag features (previous month's count)
aggregated['count_lag1'] = aggregated.groupby('grid_id')['count'].shift(1)
aggregated['count_lag3'] = aggregated.groupby('grid_id')['count'].shift(3)  # 3 months ago
aggregated['count_lag12'] = aggregated.groupby('grid_id')['count'].shift(12)  # Same month last year

# Fill NaN lags with 0 (for early months)
aggregated['count_lag1'] = aggregated['count_lag1'].fillna(0)
aggregated['count_lag3'] = aggregated['count_lag3'].fillna(0)
aggregated['count_lag12'] = aggregated['count_lag12'].fillna(0)

# Historical average count for this grid cell
grid_avg = aggregated.groupby('grid_id')['count'].transform('mean')
aggregated['grid_avg_count'] = grid_avg

print("✓ Feature engineering complete")
print(f"\nFeatures created:")
print(f"  - month_sin, month_cos (seasonality)")
print(f"  - year_normalized (trend)")
print(f"  - count_lag1, count_lag3, count_lag12 (autoregressive)")
print(f"  - grid_avg_count (historical average)")
print(f"\nDataset shape: {aggregated.shape}")

## 4. Train Baseline Prediction Model

Use Gradient Boosting to predict expected sighting counts based on:
- Location (grid_lat, grid_lon)
- Time features (year, month, seasonality)
- Historical counts (lags)

In [ ]:
# Define features and target
feature_cols = [
    'grid_lat', 'grid_lon',
    'year_normalized', 'month',
    'month_sin', 'month_cos',
    'count_lag1', 'count_lag3', 'count_lag12',
    'grid_avg_count'
]

X = aggregated[feature_cols]
y = aggregated['count']

# Split into train and test
# Use temporal split: train on earlier data, test on later data
split_year = 2010
train_mask = aggregated['year'] < split_year
test_mask = aggregated['year'] >= split_year

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print(f"Training data: {len(X_train):,} samples (before {split_year})")
print(f"Test data: {len(X_test):,} samples ({split_year} and after)")

In [ ]:
# Train Gradient Boosting model
print("Training Gradient Boosting model...")

model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=0
)

model.fit(X_train, y_train)

print("✓ Model training complete")

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluate
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"\n📊 Model Performance:")
print(f"  Train R²: {train_r2:.3f}")
print(f"  Test R²: {test_r2:.3f}")
print(f"  Train MAE: {train_mae:.2f}")
print(f"  Test MAE: {test_mae:.2f}")

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance for Sighting Count Prediction')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 most important features:")
print(feature_importance.head())

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot - Train
axes[0].scatter(y_train, y_pred_train, alpha=0.3, s=10)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Count')
axes[0].set_ylabel('Predicted Count')
axes[0].set_title(f'Training Set (R² = {train_r2:.3f})')
axes[0].grid(True, alpha=0.3)

# Scatter plot - Test
axes[1].scatter(y_test, y_pred_test, alpha=0.3, s=10)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Count')
axes[1].set_ylabel('Predicted Count')
axes[1].set_title(f'Test Set (R² = {test_r2:.3f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Compute Anomaly Scores

Calculate anomaly scores as standardized residuals (z-scores):
- Residual = Actual - Predicted
- Z-score = (Residual - Mean) / StdDev

High positive z-scores indicate more sightings than expected (anomalies).

In [ ]:
# Predict on full dataset
aggregated['predicted_count'] = model.predict(X)

# Calculate residuals
aggregated['residual'] = aggregated['count'] - aggregated['predicted_count']

# Calculate z-scores (standardized residuals)
residual_mean = aggregated['residual'].mean()
residual_std = aggregated['residual'].std()
aggregated['anomaly_score_cell'] = (aggregated['residual'] - residual_mean) / residual_std

print("✓ Anomaly scores computed")
print(f"\nAnomaly score statistics:")
print(aggregated['anomaly_score_cell'].describe())

# Find top anomalies
top_anomalies = aggregated.nlargest(10, 'anomaly_score_cell')[[
    'grid_id', 'year_month', 'count', 'predicted_count', 'anomaly_score_cell'
]]

print("\n🔥 Top 10 Anomalous Grid-Time Cells:")
print(top_anomalies.to_string(index=False))

In [ ]:
# Visualize anomaly score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(aggregated['anomaly_score_cell'], bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Anomaly Score (Z-Score)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Anomaly Scores')
axes[0].axvline(0, color='red', linestyle='--', label='Expected (0)')
axes[0].axvline(2, color='orange', linestyle='--', label='2σ threshold')
axes[0].axvline(-2, color='orange', linestyle='--')
axes[0].legend()

# Q-Q plot to check if residuals are normally distributed
stats.probplot(aggregated['anomaly_score_cell'], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot of Anomaly Scores')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Count anomalies by threshold
anomaly_threshold = 2.0  # 2 standard deviations
high_anomalies = (aggregated['anomaly_score_cell'] > anomaly_threshold).sum()
low_anomalies = (aggregated['anomaly_score_cell'] < -anomaly_threshold).sum()

print(f"\nAnomalies (|z| > {anomaly_threshold}):")
print(f"  High anomalies (more than expected): {high_anomalies:,}")
print(f"  Low anomalies (less than expected): {low_anomalies:,}")
print(f"  Total: {high_anomalies + low_anomalies:,} ({(high_anomalies + low_anomalies) / len(aggregated) * 100:.1f}%)")

## 6. Visualize Spatiotemporal Patterns

In [ ]:
# Heatmap: Average anomaly score by grid cell
grid_anomaly = aggregated.groupby(['grid_lat', 'grid_lon'])['anomaly_score_cell'].mean().reset_index()

# Create pivot table for heatmap
pivot = grid_anomaly.pivot(index='grid_lat', columns='grid_lon', values='anomaly_score_cell')

plt.figure(figsize=(16, 10))
sns.heatmap(pivot, cmap='RdBu_r', center=0, robust=True, cbar_kws={'label': 'Avg Anomaly Score'})
plt.title('Average Anomaly Score by Grid Cell (Spatial Heatmap)')
plt.xlabel('Longitude Grid')
plt.ylabel('Latitude Grid')
plt.tight_layout()
plt.show()

In [ ]:
# Time series: Average anomaly score over time
temporal_anomaly = aggregated.groupby('year')['anomaly_score_cell'].mean()

plt.figure(figsize=(14, 6))
plt.plot(temporal_anomaly.index, temporal_anomaly.values, marker='o', linewidth=2)
plt.axhline(0, color='red', linestyle='--', label='Expected baseline')
plt.xlabel('Year')
plt.ylabel('Average Anomaly Score')
plt.title('Temporal Trend of Anomaly Scores')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 7. Export Results

Save the aggregated data with anomaly scores for use in:
- Task 2.5: Per-report anomaly scoring (merge back)
- Frontend: Visualization

In [ ]:
# Select columns to export
export_cols = [
    'grid_id', 'grid_lat', 'grid_lon',
    'year', 'month', 'year_month',
    'count',
    'predicted_count',
    'residual',
    'anomaly_score_cell'
]

output_path = Path('../data/processed/grid_time_anomalies.parquet')
aggregated[export_cols].to_parquet(output_path, index=False)

print(f"✓ Exported {len(aggregated):,} grid-time cells to:")
print(f"  {output_path}")
print(f"\n✅ Task 2.3 Complete!")
print(f"\nNext steps:")
print(f"  1. Review the anomaly scores and visualizations above")
print(f"  2. Proceed to Task 2.4: Text embeddings & clustering")
print(f"  3. Or explore specific anomalous regions in detail")